In [22]:
import numpy as np
from sklearn.impute import KNNImputer
import pandas as pd
import os

In [23]:
# Set config
outputPath = '.'
pipeline_config = {
    'feature_eng_version': '0.1',
    'model_version': '0.1',
    'info_text': None
}

In [24]:
pd.set_option('display.max_columns', 100)

# VIF

In [25]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

df = pd.DataFrame(
    {'a': [1, 1, 2, 3, 4],
     'b': [2, 2, 3, 2, 1],
     'c': [4, 6, 7, 8, 9],
     'd': [4, 3, 4, 5, 4]}
)

X = add_constant(df)
pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

/home/srinesh/python/code-snippets/env/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


const    136.875
a         22.950
b          3.000
c         12.950
d          3.000
dtype: float64

# Data Imputation

In [2]:
nan = np.nan
X = [[1, 2, nan], [3, 4, True], [nan, 6, True], [8, 8, False]]
imputer = KNNImputer(n_neighbors=2, weights="uniform")
imputer.fit_transform(X)

array([[1. , 2. , 1. ],
       [3. , 4. , 1. ],
       [5.5, 6. , 1. ],
       [8. , 8. , 0. ]])

# Ordinal Encoding

In [51]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X = [['Male','Female']]

In [12]:
enc.fit_transform(X)

array([[0., 0.]])

## One-Hot Encoding

In [46]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False, handle_unknown='error', drop='if_binary')

flowers = pd.DataFrame({
    'color' : ['red', 'green', 'red', 'green', 'red', 'green', 'red', 'green', 'blue', 'blue'],
    'pretty': ['p', 'p', 'n', 'u', 'p', 'u', 'n', 'p', 'n', 'p'],
    'native': [True, True, True, False, False, True, False, True, False, False],
    'height': [4,9,4,8,4,7,4,7.5,20,19],
    'petals': [3,9,1,8,1,10,2,8,50,47],
    'days'  : [6,16,7,15,8,17,5,12,40,45]
})


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(flowers.drop('days', 1), flowers['days'],
                                                    test_size=.2, random_state=40)

/tmp/ipykernel_1735/2193963073.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train, X_test, y_train, y_test = train_test_split(flowers.drop('days', 1), flowers['days'],


In [49]:
ohe.fit_transform(X_train[['color', 'pretty', 'native']])

array([[1., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 1.],
       [0., 0., 1., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 1., 0., 1.],
       [0., 1., 0., 0., 0., 1., 1.],
       [0., 1., 0., 0., 1., 0., 1.],
       [0., 0., 1., 1., 0., 0., 0.]])

In [50]:
ohe.get_feature_names()

array(['x0_blue', 'x0_green', 'x0_red', 'x1_n', 'x1_p', 'x1_u', 'x2_True'],
      dtype=object)

In [39]:
# # Get all categoric columns
# categ = [col for col in X_train.columns if X_train[col].dtypes == 'O']

# o3 = OneHotEncoder(sparse=False)
# o3.fit_transform(X_train[categ])

# Dump data into a pickle file

In [61]:
import pickle
modelling = {}
modelling['config'] = pipeline_config
modelling['Data'] = {'Xtrain': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test':y_test}
pickle.dump(modelling, open(os.path.join(outputPath, 'data_v1.pkl'), 'wb'))

In [66]:
infile = open(os.path.join(outputPath, 'data_v1.pkl'), 'rb')
new_dict = pickle.load(infile, encoding='utf-8')

In [1]:
#d

In [10]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(probability=True))])
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)

0.88

In [17]:
from sklearn.metrics import roc_auc_score

In [18]:
roc_auc_score(y_test, pipe.predict_proba(X_test)[:,1])

0.8974358974358974